残差网络
===

# 1.传统网络的问题
传统的神经网络中，随着网络层数的增加，网络发生了退化(degradation)的现象：随着网络层数的增多，训练集loss逐渐下降，然后趋于饱和，当你再增加网络深度的话，训练集loss反而会增大。

注意这并不是过拟合，因为在过拟合中训练loss是一直减小的。当网络退化时，浅层网络能够达到比深层网络更好的训练效果，这时如果我们把低层的特征传到高层，那么效果应该至少不比浅层的网络效果差，

或者说如果一个VGG-100网络在第98层使用的是和VGG-16第14层一模一样的特征，那么VGG-100的效果应该会和VGG-16的效果相同。

所以，我们可以在VGG-100的98层和14层之间添加一条直接映射(Identity Mapping)来达到此效果。基于这种使用直接映射来连接网络不同层直接的思想，残差网络应运而生

# 2.残差网络
## 2.1.残差块
残差网络是由一系列残差块组成的（图1）。一个残差块可以用表示为：
$$x_{l+1}= x_l+\mathcal{F}(x_l, {W_l})$$
残差块分成两部分直接映射部分和残差部分。$h(x_l)$是直接映射，反应在图1中是左边的曲线；$\mathcal{F}(x_l, {W_l})$是残差部分，一般由两个或者三个卷积操作构成，即图1中右侧包含卷积的部分。
![image](Images/03/01/04_001.png)
图中的'Weight‘在卷积网络中是指卷积操作，’addition‘是指单位加操作。在卷积网络中，$x_l$可能和$x_{l+1}$的Feature Map的数量不一样，这时候就需要使用$1 \times 1$卷积进行升维或者降维。这时，残差块表示为：
$$x_{l+1}= h(x_l)+\mathcal{F}(x_l, {W_l})$$
![image](Images/03/01/04_002.png)
其中$h(x_l) = W'_lx$。其中$W'_l$是$1 \times 1$卷核，是实验结果$1 \times 1$卷积对模型性能提升有限，所以一般是在升维或者降维时才会使用。一般，这种版本的残差块叫做resnet_v1

In [1]:
import torch


class Res_Block_V1(torch.nn.Module):
    def __init__(self, input_filter, output_filter):
        super(Res_Block_V1, self).__init__()
        self.input_filter = input_filter
        self.output_filter = output_filter
        
        self.conv1 = torch.nn.Conv2d(in_channels=input_filter, out_channels=output_filter, kernel_size=3, stride=1)
        self.norm1 = torch.nn.BatchNorm2d(num_features=output_filter)
        self.conv2 = torch.nn.Conv2d(in_channels=output_filter, out_channels=output_filter, kernel_size=3, stride=1)
        self.norm2 = torch.nn.BatchNorm2d(num_features=output_filter)
        self.relu  = torch.nn.ReLU()
        
        self.otherConv = torch.nn.Conv2d(in_channels=output_filter, out_channels=output_filter, kernel_size=1, stride=1)
        
    def forward(self, input):
        x = self.conv1(input)
        x = self.norm1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.norm2(x)
        
        if self.input_filter == self.output_filter:
            identity = input
        else:
            identity = self.otherConv(x)
        
        x += identity
        x = self.relu(x)
        return x

## 2.2.残差网络
残差网络的搭建分为两步：
- 使用VGG公式搭建Plain VGG网络
- 在Plain VGG的卷积网络之间插入Identity Mapping，注意需要升维或者降维的时候加入$1 \times 1$卷积。

在实现过程中，一般是直接stack残差块的方式。

In [2]:
class Resnet(torch.nn.Module):
    def __init__(self):
        super(Resnet, self).__init__()
        self.linear_infeature = 64
        self.conv1 = torch.nn.Conv2d(in_channels=3, out_channels=16, stride=1, kernel_size=3)
        self.relu = torch.nn.ReLU()
        self.block1 = Res_Block_V1(16, 16)
        self.block2 = Res_Block_V1(16, 32)
        self.fc = torch.nn.Linear(in_features=self.linear_infeature, out_features=10)
        self.softmax = torch.nn.Softmax()
        
    def forward(self, input):
        x = self.conv1(input)
        x = self.block1(x)
        x = self.block2(x)
        x = x.view(-1, self.linear_infeature)
        x = self.fc(x)
        x = self.softmax(x)
        
        return x

## 2.3.为什么叫做残差网络
在统计学中，残差和误差是非常容易混淆的两个概念。
- 误差是衡量观测值和真实值之间的差距
- 残差是指预测值和观测值之间的差距

网络的一层通常可以看做$y=H(x)$, 而残差网络的一个残差块可以表示为$H(x)=F(x)+x$，也就是$F(x) = H(x)-x$，在单位映射中，$y=x$便是观测值，

而$H(x)$是预测值，所以$F(x)$便对应着残差，因此叫做残差网络。

## 2.4.Resnet V2版本
经过试验之后发现，激活函数移动到残差部分可以提高模型的精度

In [ ]:
import torch


class Res_Block_V2(torch.nn.Module):
    def __init__(self, input_filter, output_filter):
        super(Res_Block_V1, self).__init__()
        self.input_filter = input_filter
        self.output_filter = output_filter
        
        self.norm0 = torch.nn.BatchNorm2d(num_features=input_filter)
        self.relu  = torch.nn.ReLU()
        
        self.conv1 = torch.nn.Conv2d(in_channels=input_filter, out_channels=output_filter, kernel_size=3, stride=1)
        self.norm1 = torch.nn.BatchNorm2d(num_features=output_filter)
        
        self.conv2 = torch.nn.Conv2d(in_channels=output_filter, out_channels=output_filter, kernel_size=3, stride=1)
        
        
        
        self.otherConv = torch.nn.Conv2d(in_channels=output_filter, out_channels=output_filter, kernel_size=1, stride=1)
        
    def forward(self, input):
        x = self.norm0(input)
        x = self.relu(x)
        
        x = self.conv1(input)
        x = self.norm1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.norm2(x)
        
        if self.input_filter == self.output_filter:
            identity = input
        else:
            identity = self.otherConv(x)
        
        x += identity
        x = self.relu(x)
        return x


class Resnet_V2(torch.nn.Module):
    def __init__(self):
        super(Resnet, self).__init__()
        self.linear_infeature = 64
        self.conv1 = torch.nn.Conv2d(in_channels=3, out_channels=16, stride=1, kernel_size=3)
        self.relu = torch.nn.ReLU()
        self.block1 = Res_Block_V2(16, 16)
        self.block2 = Res_Block_V2(16, 32)
        self.norm1 = torch.nn.BatchNorm2d(num_features=32)
        self.fc = torch.nn.Linear(in_features=self.linear_infeature, out_features=10)
        self.softmax = torch.nn.Softmax()
        
    def forward(self, input):
        x = self.conv1(input)
        x = self.block1(x)
        x = self.block2(x)
        x = self.norm1(x)
        x = x.view(-1, self.linear_infeature)
        x = self.fc(x)
        x = self.softmax(x)
        
        return x